This notebook is used for pre processing the data for evaluation. Mainly adding attention information.

In [ ]:
import torch
import pickle
import json
import subprocess
import os
import os.path as osp
import string

In [12]:
#make a list of all the sentences in the model data
#and get an incremental list to pass into the parser
#and make a list of the length of each incremental unit
def make_model_inc(model):
    len_increments=[]
    sents_model_split=[]

    for i in range(len(model)):
        sents_model_split.append(model[i][3])

    inc_sents=[]
    for entry in sents_model_split:
        hold_sent=entry.split()
        
        #remove all punctuation since the parser cant handle it
        hold_sent = [''.join(c for c in s if c not in string.punctuation) for s in hold_sent]
        
       
        if '' in hold_sent:
            hold_sent= [element for element in hold_sent if element != '']
     
        len_increments.append(len(hold_sent))

        for i in range(len(hold_sent)):
            
           
            inc_sents.append(" ".join(hold_sent[0:i+1]))


    return (inc_sents,len_increments)

In [4]:
dataset_list=["unc","gref","gref_umd","unc+"]
split_list_test=["_testB","_train","_val","_test","_testA"]


In [14]:
#load all validation and test split datasets
#and save incremental datalists
for dataset in dataset_list:
    for split in split_list_test:
        try:
            model=torch.load("/home/users/fschreiber/project/ReSC/data/"+dataset+"/"+dataset+split+".pth")
            print(dataset+split)
            inc_sents,len_inc=make_model_inc(model)
            
            try:
                with open(r"/home/users/fschreiber/project/incremental_pickles/"+dataset+split+".p", "wb") as output_file:
                    pickle.dump(inc_sents, output_file)
            except Exception as e:
                print("something went wrong in saving the incremental pickles")
                print(e)

            try:
                with open(r"/home/users/fschreiber/project/incremental_pickles/length_incremental_units/"+dataset+split+"_length_unit.p", "wb") as output_file:
                    pickle.dump(len_inc, output_file)
            except Exception as e:
                print("something went wrong in saving the incremental lists pickles")
                print(e)

        except(FileNotFoundError):
            pass
            

unc_testB
unc_train
unc_val
unc_testA
gref_train
gref_val
gref_umd_train
gref_umd_val
gref_umd_test
unc+_testB
unc+_train
unc+_val
unc+_testA


In [8]:
#this would work in theory but the parser is very slow
#its better to run the parses manually 

#for dataset in dataset_list:
#    for split in split_list_test:
#        try:
#            subprocess.run("python ./refer-parser2/parse_sents_model.py --splitBy "+dataset+ " --partition "+split,shell=True)
#        except (FileNotFoundError):
#            pass

In [2]:
dataset_list=["gref","gref_umd"]
split_list_test=["_train","_test","_val"]

In [4]:
#to work with the attention parser the parsed lists need to be formatted
# to match the attention parser sturcutre

def prepare_for_att_parser(noisy_list):
    
    for entry in range(len(noisy_list)):
        
        #change "sentences" to "parse"
        noisy_list[entry]["parse"]=noisy_list[entry]["sentences"][0]
        del(noisy_list[entry]["sentences"])

        #the indexdependencies need to be split up and written into dependencies
        dep_list=[]
        for j in range(len(noisy_list[entry]["parse"]["indexeddependencies"])):
            hold=[]
            for i in range(len(noisy_list[entry]["parse"]["indexeddependencies"][j])):
                hold=hold+noisy_list[entry]["parse"]["indexeddependencies"][j][i].split("-")
                
            dep_list.append(hold)
        noisy_list[entry]["parse"]["dependencies"] = dep_list  
    return noisy_list

In [7]:
#clean the parsed lists for attention parsing
for dataset in dataset_list:
    for split in split_list_test:
        try:
           
            with open("/home/users/fschreiber/project/parses/"+dataset+split+"/sents.json") as f:
                noisy_list = json.load(f)
                print("clean set: ",dataset,split)
            
            clean_list=prepare_for_att_parser(noisy_list)

            try:
                with open("/home/users/fschreiber/project/att_prep/"+dataset+split+".json", 'w') as f:
                    json.dump(clean_list, f)
            except Exception as e:
                print("something went wrong with cleaning the parsed list and saving them")
                print(e)

            if dataset=="referit":
                subprocess.run("python ./refer-parser2/parse_atts.py --dataset referit --splitBy " +dataset+ " --partition " +split,shell=True)  

            else:
                subprocess.run("python ./refer-parser2/parse_atts.py --dataset refcoco --splitBy " +dataset+ " --partition " +split,shell=True) 
                
               
        except (FileNotFoundError):
            print("not found"+dataset+split)

clean set:  gref _train
1/713953 has been decomposed into attributes r1-r8.
101/713953 has been decomposed into attributes r1-r8.
201/713953 has been decomposed into attributes r1-r8.
301/713953 has been decomposed into attributes r1-r8.
401/713953 has been decomposed into attributes r1-r8.
501/713953 has been decomposed into attributes r1-r8.
601/713953 has been decomposed into attributes r1-r8.
701/713953 has been decomposed into attributes r1-r8.
801/713953 has been decomposed into attributes r1-r8.
901/713953 has been decomposed into attributes r1-r8.
1001/713953 has been decomposed into attributes r1-r8.
1101/713953 has been decomposed into attributes r1-r8.
1201/713953 has been decomposed into attributes r1-r8.
1301/713953 has been decomposed into attributes r1-r8.
1401/713953 has been decomposed into attributes r1-r8.
1501/713953 has been decomposed into attributes r1-r8.
1601/713953 has been decomposed into attributes r1-r8.
1701/713953 has been decomposed into attributes r1-r8

In [2]:
dataset_list=["gref_umd"]
split_list_test=["_train"]

In [3]:
#combine the image data with the incremental attention data to make the evaluation set
for dataset in dataset_list:
    for split in split_list_test:
        try:
            model = torch.load("/home/users/fschreiber/project/ReSC/data/"+dataset+"/"+dataset+split+".pth")
            #print("found" ,dataset+"/"+dataset+split,".pth")
            
            with open("/home/users/fschreiber/project/parsed_attention/"+dataset+split+"/sents.json") as f:
                pars_inc_model=json.load(f)
            print("found",dataset,split)

          
            #makes a data set containing incremental sentences and their pictures
            inc_model=[]
            for i in range(len(model)):
                hold_sent=model[i][3].split()

                #remove all punctuation since the parser cant handle it
                hold_sent = [''.join(c for c in s if c not in string.punctuation) for s in hold_sent]
                
                if '' in hold_sent:
                    hold_sent= [element for element in hold_sent if element != '']
                    
                for j in range(len(hold_sent)):
                    
                    tup=(model[i][0],model[i][1],model[i][2]," ".join(hold_sent[0:j+1]))
                    inc_model.append(tup)
                     
            

            #match the attention attributes to the images by comapring the incremental "sentences"
            comp_data=[]
            for i in range(len(pars_inc_model)):
                if pars_inc_model[i]["parse"]["text"] != inc_model[i][3]:
                    print(i,"something is not matching")
                    pass
                else:
                    hold=(pars_inc_model[i]["atts"],)
                    comp_data.append(inc_model[i]+hold)

        
            #save the dataset
            if not osp.isdir('/home/users/fschreiber/project/ready_inc_data/'+dataset):
                os.makedirs('/home/users/fschreiber/project/ready_inc_data/' +dataset)

            torch.save(comp_data, '/home/users/fschreiber/project/ready_inc_data/' +dataset+"/"+dataset+split+".pth")
       
        except (FileNotFoundError):
            pass

found gref _train
found gref _val
found gref_umd _train
found gref_umd _test
found gref_umd _val
